# HypoClock

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.HypoClock)

class HypoClock(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Compute mean beta per sample, excluding missing (-1) values.
        """
        means = []
        for row in x:
            filtered_row = row[row != -1]
            if len(filtered_row) > 0:
                mean = torch.mean(filtered_row)
            else:
                mean = torch.tensor(float("nan"), device=x.device, dtype=x.dtype)
            means.append(mean)
        return torch.vstack(means)

    def postprocess(self, x):
        return 1 - x



In [3]:
model = pya.models.HypoClock()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'hypoclock'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2018
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Zhou, Wanding, et al. \"DNA methylation loss in late-replicating domains is linked to mitotic cell division.\" Nature Genetics 50, no. 4 (2018): 591-602."
model.metadata["doi"] = "https://doi.org/10.1038/s41588-018-0073-4"
model.metadata["research_only"] = None
model.metadata["notes"] = "HypoClock score is 1 - mean beta across 678 solo-WCGW CpGs; reference values are -1 to ignore missing."

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://raw.githubusercontent.com/aet21/EpiMitClocks/master/data/dataETOC3.rda"
supplementary_file_name = "dataETOC3.rda"
os.system(f"curl -L -o {supplementary_file_name} {supplementary_url}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30851  100 30851    0     0   425k      0 --:--:-- --:--:-- --:--:--  430k


0

## Load features

#### From R data

In [6]:
r_cmd = (
    "load('dataETOC3.rda'); "
    "df <- data.frame(cpg=dataETOC3.l[[4]]); "
    "write.csv(df, 'hypoclock_cpgs.csv', row.names=FALSE)"
)
os.system(f"Rscript -e \"{r_cmd}\"")

features_df = pd.read_csv('hypoclock_cpgs.csv')
model.features = features_df['cpg'].tolist()

assert len(model.features) == 678, f"Expected 678 CpGs, got {len(model.features)}"

## Load weights into base model

In [8]:
weights = torch.tensor([1.0]).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=1)

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [10]:
model.reference_values = [-1]*len(model.features)

## Load preprocess and postprocess objects

In [11]:
model.preprocess_name = "mean"
model.preprocess_dependencies = None

In [12]:
model.postprocess_name = "one_minus"
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Zhou, Wanding, et al. "DNA methylation loss in late-replicating '
             'domains is linked to mitotic cell division." Nature Genetics 50, '
             'no. 4 (2018): 591-602.',
 'clock_name': 'hypoclock',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s41588-018-0073-4',
 'notes': 'HypoClock score is 1 - mean beta across 678 solo-WCGW CpGs; '
          'reference values are -1 to ignore missing.',
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2018}
reference_values: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]... [Total elements: 678]
preprocess_name: 'mean'
preprocess_dependencies: None
postprocess_name: 'one_minus'
postprocess_dependencies: None
features: ['cg01689796', 'cg177531

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[0.9638],
        [0.9622],
        [1.0423],
        [1.0675],
        [1.0330],
        [0.9733],
        [0.9686],
        [1.0028],
        [0.9955],
        [0.9212]], dtype=torch.float64, grad_fn=<RsubBackward1>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: dataETOC3.rda
Deleted file: hypoclock_cpgs.csv
